# Load Packages and Data

In [5]:
using CSV, DataFrames, JuMP, Gurobi;

In [98]:
# set file paths in case we move this file or the data later
points_path = "../../data/final/single_solution"
solution_path ="../../data/processed/actual_points"

"../../data/processed/actual_points"

## Join selected team with actual points

In [99]:
function join_by_week(week)
df1 = CSV.read("$points_path/week_$week.csv", DataFrame) # read in the csv for points scored each week
df2 = CSV.read("$solution_path/gw$week.csv", DataFrame) # read in the csv for the team we selected
rename!(df2, "Name" => "Player") # rename column to help with join
result_df = leftjoin(df1, df2[:, [:Player, :Price, :TP]], on = :Player) # join on player
    return result_df
end

join_by_week (generic function with 1 method)

In [97]:
join_by_week(2)

Row,Player,Squad,Team,Captain,TripCaptain,BB,WC,FH,TTr,Price,TP
,String15,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Float64?,Int64?
1,Saka,1.0,1.0,0.0,0,0.0,0.0,0.0,0.0,8.5,3
2,Turner,1.0,0.0,0.0,0,0.0,0.0,0.0,0.0,4.0,2
3,Cash,1.0,1.0,0.0,0,0.0,0.0,0.0,0.0,4.5,6
4,Martinez,1.0,1.0,0.0,0,0.0,0.0,0.0,0.0,5.0,8
5,Mbeumo,1.0,1.0,0.0,0,0.0,0.0,0.0,0.0,7.0,16
6,Enzo,1.0,1.0,0.0,0,0.0,0.0,0.0,0.0,5.0,0
7,N.Jackson,1.0,1.0,0.0,0,0.0,0.0,0.0,0.0,7.0,1
8,Salah,1.0,1.0,1.0,0,0.0,0.0,0.0,0.0,12.5,5
9,Andersen,1.0,0.0,0.0,0,0.0,0.0,0.0,0.0,4.0,missing


## Calculate points added by each player for a given week

In [100]:
function calc_points(week)
    # get the joined df for that week
    df = join_by_week(week) 
    # get total points added for that week if selected
    df[!, :PointsAdded] = Float64.(ifelse.(df.Team .+ df.BB .>= 1, (1 .+ df.Captain .+ df.TripCaptain) .* df.TP, 0)) 
    # get the sum of these points
    tot_pts = sum(df[!, :PointsAdded])
    return tot_pts
end

calc_points (generic function with 1 method)

## Calculate points over a given range of weeks

In [101]:
function points_over_range(starting_week, ending_week)
    tot_pts = 0
    for week in starting_week:ending_week # sum for range of weeks
        tot_pts += calc_points(week)
    end
    return tot_pts
end

points_over_range (generic function with 1 method)

In [95]:
points_over_range(1,9)

600.5

## Calculate cost by each player for a given week

In [103]:
function calc_cost(week)
    # get the joined df for that week
    df = join_by_week(week) 
    # get total cost added for that week if selected
    tot_cost = sum(df[!, :Price])
    return tot_cost
end

calc_cost (generic function with 1 method)

## Calculate points over a given range of weeks

In [104]:
function cost_over_range(starting_week, ending_week)
    tot_cost = 0
    for week in starting_week:ending_week # sum for range of weeks
        tot_cost += calc_cost(week)
    end
    return tot_cost
end

cost_over_range (generic function with 1 method)

In [114]:
cost_over_range(1,9)

896.5